In [43]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [44]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [47]:
paises_este_europa = {
    "BY": "Bielorrusia",
    "BG": "Bulgaria",
    "CZ": "Chequia",
    "SK": "Eslovaquia",
    "HU": "Hungría",
    "MD": "Moldavia",
    "PL": "Polonia",
    "RO": "Rumanía",
    "UA": "Ucrania"}

In [48]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [49]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [50]:
p_este = extraccion_por_paises(paises_este_europa, '2015-2019')

In [51]:
dic_este = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [52]:
filtro_de_datos(p_este, dic_este)

In [53]:
dic_este

{'nombre': ['Супермаркет',
  "Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson)",
  'Ханнамонтана',
  'Holding Back the Years (Live at Ziggo Dome, Amsterdam)',
  'Нон стоп',
  'Say You Love Me (Live at Ziggo Dome, Amsterdam)',
  'мальчик',
  'For Your Babies (Live at Ziggo Dome, Amsterdam)',
  'Это любовь',
  'So Beautiful (Live at Ziggo Dome, Amsterdam)',
  'Все хотят меня поцеловать',
  'Your Mirror (Live at Ziggo Dome, Amsterdam)',
  'ПОРНХАБ КРИСМАС КЛАБ',
  'Someday in My Life (Live at Ziggo Dome, Amsterdam)',
  'Пьяный дождь',
  'Ee Khalbitha (From "IDI")',
  'Тмстс',
  'A New Flame (Live at Ziggo Dome, Amsterdam)',
  'Райские цветы',
  '言葉のいらない約束',
  'Falling Down - Bonus Track',
  'Do It Like You Mean It (feat. Yoel 905)',
  'Крепче',
  'Stronger Together',
  'Утро',
  'Kapitel 1 / Frühling',
  'Дорадура',
  'Inquilina - Ao Vivo',
  'Танцы',
  'God Is Gangsta',
  'Горы по колено',
  'Győzni Jöttünk',
  'Любовь HD1080',
  'Von Heinz zu Heinz (

Llamada a API Last.FM y filtro para conseguir el género

In [54]:
p_este_last = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist=Sasha&track=here+she+comes+again&api_key={API_KEY}&autocorrect=1&format=json")

In [55]:
p_este_last.json()['toptags']['tag'][0]['name']

'pop'

In [56]:
lista_nombres_este = list(dic_este.values())[0]
lista_artistas_este = list(dic_este.values())[2]

In [57]:
def llamada_lastfm(lista_nombres, lista_artistas,datos):
    for i in range(len(lista_nombres)):
        try:
            nombre = str(lista_nombres[i]).replace(' ', '+')
            artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
            llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
            try:
                genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
                datos['genero'].append(genero_cancion)
            except KeyError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
                
            except IndexError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no tiene tags")
            
        except:
            datos['genero'].append("NULL")
            print(f"Hay problemas con la cación {lista_nombres[i]}")

In [58]:
llamada_lastfm(lista_nombres_este, lista_artistas_este, dic_este)

La canción Greenwood, Philip Selway, Edward O'Brien, Colin Greenwood: Pyramid Song (Arr. Lawson) no está registrada en lastfm
La canción Ханнамонтана no tiene tags
La canción Holding Back the Years (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Нон стоп no tiene tags
La canción Say You Love Me (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción For Your Babies (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Это любовь no tiene tags
La canción So Beautiful (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Все хотят меня поцеловать no tiene tags
La canción Your Mirror (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Someday in My Life (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Пьяный дождь no tiene tags
La canción Ee Khalbitha (From "IDI") no está registrada en lastfm
La canción Тмстс no tiene tags
La canción A New Flame (Live at Ziggo Dome, Amsterdam) no tiene tags
La canción Райские цветы no tiene tags
La canción Falling Down - Bonus

In [59]:
df_este = pd.DataFrame(dic_este)

In [60]:
df_este.index.name="id"

In [61]:
df_este

,nombre,popularity,artista,pais,genero
id,,,,,
0,Супермаркет,57,Poshlaya Molly,BY,drugs
1,"Greenwood, Philip Selway, Edward O'Brien, Coli...",3,"Jonny Greenwood, Thom Yorke, Philip Selway, Ed...",BY,NULL
2,Ханнамонтана,56,Poshlaya Molly,BY,NULL
3,"Holding Back the Years (Live at Ziggo Dome, Am...",2,Simply Red,BY,NULL
4,Нон стоп,55,Poshlaya Molly,BY,NULL
...,...,...,...,...,...
445,Du bist schön - Live,0,Alligatoah,UA,NULL
446,Кольорова,46,Скрябін,UA,Ukrainian
447,Verdi: Il Trovatore - Tacea la notte placida,0,"Giuseppe Verdi, Sonya Yoncheva, Massimo Zanett...",UA,NULL


In [62]:
df_este.to_csv("paises_este_2015_2019.csv")